In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


def ipu(datafile, constrains):
    df = pd.read_csv(datafile)
    ipf = constrains
    df.rename(columns={df.columns[0]: 'SERIALNO'}, inplace=True)
    e = np.finfo('float').eps
    for i in ipf:
        if i == 0:
            i = e
    # w = np.array(df['weights'], dtype=np.float32)
    w=np.ones(len(df))
    df = df.drop(['SERIALNO'], axis=1)
    # df = df.drop(['weights'], axis=1)
    ws = np.zeros(len(df))
    i = 0

    for col  in df.columns:
        x = 0
        for j in range(len(df)) :
            # print(w[j], df[col][j])
            x += w[j]*df[col][j]
        ws[i] = x
        i += 1
    print('weighted sums = ', ws)

    delta = sum((abs(ws-ipf))/ipf)
    print(delta)

    a = df.to_numpy(dtype=np.float32)
    # print(a.shape)
    # print(df.shape)
    print(a)
    print(type(a[1][1]))

    w1 = open("ipu-weights.txt", "w")
    w2 = open("ipu-weighted_sums.txt", "w")
    
    t=1

    while(True):

        delta_prev = delta

        w1.write('iteration'+str(t)+'\n')
        w2.write('iteration'+str(t)+'\n')

        for i in range(df.shape[1]):
            pws = ws[i]
            ws[i] = ipf[i]
            multiplier = ws[i]/pws
            # print(multiplier)
        
            for j in range(df.shape[0]):
                if(a[j][i] == 0):
                    continue
                w[j] *= multiplier
        
            for k in range(df.shape[1]):
                x = 0
                for j in range(df.shape[0]):
                    x += a[j][k]*w[j]
                ws[k] = x
        
            w1.write('weights'+str(i)+str(w)+'\n')
            w2.write("weighted sum"+str(i+1)+str(ws)+'\n')

        delta = sum((abs(ws-ipf))/ipf)
        if(abs(delta_prev-delta) < .0001):
            break
        t+=1




In [2]:
f1=open('person_constrains.txt','r')
f1=f1.read().split()
person=list(f1)
f1=open('hhold_constrains.txt','r')
f1=f1.read().split()
hhold=list(f1)
ipf=np.array(hhold+person)
# x = ipu('binary_matrix.csv', ipf)
len(ipf)

3292

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# def ipu(datafile, constrains):
datafile='binary_matrix.csv'
df = pd.read_csv(datafile)
# ipf = constrains
df.rename(columns={df.columns[0]: 'SERIALNO'}, inplace=True)
e = np.finfo('float').eps
for i in ipf:
    if i == 0:
        i = e
# w = np.array(df['weights'], dtype=np.float32)
w=np.ones(len(df))
df = df.drop(['SERIALNO'], axis=1)
# df = df.drop(['weights'], axis=1)
ws = np.zeros(len(ipf))
i = 0

for col  in df.columns:
    x = 0
    for j in range(len(df)) :
        # print(w[j], df[col][j])
        x += w[j]*df[col][j]
    ws[i] = x
    i += 1
print('weighted sums = ', ws)




C:\Users\abubabu\AppData\Local\Temp\ipykernel_40068\2858024573.py:8: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(datafile)


weighted sums =  [  0.   0.   0. ...  25.   0. 740.]


In [4]:


# len(ws),len(ipf),len(df)

ws.shape != ipf.shape
ws1=ws.copy()
ipf1=ipf.copy()

In [5]:
# def print_unique(arr):
#     # Get unique dtypes and their counts
#     unique_dtypes, dtype_counts = np.unique(arr, return_counts=True)

#     # Print the unique dtypes and their counts
#     for dtype, count in zip(unique_dtypes, dtype_counts):
#         print(f"Dtype: {dtype}, Count: {count}")
# print_unique(ipf)
# print_unique(ws)
ipf=ipf.astype(np.float32)

In [6]:
ws = np.where(ws is None, 0, ws)
ipf = np.where(ipf is None, 0, ipf)
abs_diff = np.abs(ws - ipf)
# delta = np.sum(np.abs(ws - ipf) / ipf) # 0 values 
delta = np.sum(np.abs(ws - ipf))
# delta = sum((abs(ws-ipf))/ipf)
print(delta)

162119299.5325384


In [7]:
df.shape[1]

3292

In [8]:
ws[1]/ws

C:\Users\abubabu\AppData\Local\Temp\ipykernel_40068\2132476098.py:1: RuntimeWarning: invalid value encountered in true_divide
  ws[1]/ws


array([nan, nan, nan, ...,  0., nan,  0.])

In [10]:
a = df.to_numpy(dtype=np.float32)
# print(a.shape)
# print(df.shape)
print(a)
print(type(a[1][1]))

w1 = open("ipu-weights.txt", "w")
w2 = open("ipu-weighted_sums.txt", "w")

t=1

for t in range(500):
    # print(t)
    delta_prev = delta

    w1.write('iteration'+str(t)+'\n')
    w2.write('iteration'+str(t)+'\n')

    for i in range(df.shape[1]):
        pws = ws[i]
        ws[i] = ipf[i]
        multiplier = ws[i]/pws
        # print(multiplier)
    
        for j in range(df.shape[0]):
            if(a[j][i] == 0):
                continue
            w[j] *= multiplier
    
        for k in range(df.shape[1]):
            x = 0
            for j in range(df.shape[0]):
                x += a[j][k]*w[j]
            ws[k] = x
    
        w1.write('weights'+str(i)+str(w)+'\n')
        w2.write("weighted sum"+str(i+1)+str(ws)+'\n')

    delta = sum((abs(ws-ipf)))
    if(abs(delta_prev-delta) < 100):
        break
    t+=1
    print('iteration ->',t,';','delta ->',delta)


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
<class 'numpy.float32'>


C:\Users\abubabu\AppData\Local\Temp\ipykernel_33500\1526317413.py:22: RuntimeWarning: invalid value encountered in double_scalars
  multiplier = ws[i]/pws


In [ ]:
xx=pd.read_csv('binary_matrix.csv')
# print(xx.columns)
xx.rename(columns={xx.columns[0]: 'SERIALNO'}, inplace=True)
for idx, row in xx.iterrows():
    if row['SERIALNO']=='2017001221812':
        print(row.dtypes)
        break


C:\Users\abubabu\AppData\Local\Temp\ipykernel_25344\2025125891.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  xx=pd.read_csv('binary_matrix.csv')


object
